In [1]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.1.3
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [8dfed614] +

In [0]:
using CUDAnative, CUDAdrv, CuArrays, BenchmarkTools, Test

In [48]:
function parallel_red!(a)
  idx=threadIdx().x + (blockIdx().x-1)*blockDim().x
  if idx <= size(a,1)
  step=1
  while step < blockDim().x   #size(a,1)
    @inbounds temp=a[idx]
    if(idx % 1024 > step || idx % 1024==0)
      temp=a[idx]+a[idx-step]       # using @inbounds here leads to test fail
    end
   sync_threads()
    @inbounds a[idx]=temp
  step*=2
  end
  if(threadIdx().x==blockDim().x && idx!=size(a,1))
  #@cuprintf("%ld\n",idx)
  @inbounds @atomic a[size(a,1)]+=a[idx]
  end

 end
return nothing
end

parallel_red! (generic function with 1 method)

In [49]:
n=10000
h_a=rand(n)

10000-element Array{Float64,1}:
 0.14714067596544544 
 0.9685932647201392  
 0.8289172479035676  
 0.2404610794571589  
 0.7314396702590722  
 0.7113536061164283  
 0.023286003661288524
 0.570298742958032   
 0.8308083048299664  
 0.9224917555232828  
 0.5262472118759918  
 0.30524560849678606 
 0.3198130086054647  
 ⋮                   
 0.9676667239936336  
 0.6711975280432456  
 0.9149832520873205  
 0.01685754782768356 
 0.8958338627259914  
 0.32309938610343103 
 0.18200116927003895 
 0.3544937777365278  
 0.29013871732625285 
 0.2632197315594995  
 0.08916474828985033 
 0.5963152696131351  

In [0]:
d_a=CuArray(h_a)
@cuda blocks=Int(ceil(n/1024)) threads=1024 parallel_red!(d_a)

In [51]:
h_a1=Array(d_a)
h_a1

10000-element Array{Float64,1}:
    0.14714067596544544
    1.1157339406855846 
    1.9446511885891522 
    2.185112268046311  
    2.9165519383053837 
    3.6279055444218113 
    3.6511915480831    
    4.221490291041132  
    5.052298595871099  
    5.974790351394381  
    6.501037563270373  
    6.806283171767159  
    7.126096180372624  
    ⋮                  
  380.70847405381073   
  381.3796715818539    
  382.2946548339413    
  382.3115123817689    
  383.20734624449494   
  383.53044563059836   
  383.7124467998684    
  384.06694057760495   
  384.3570792949312    
  384.62029902649067   
  384.7094637747806    
 4993.198149163269     

In [52]:
function seq_red(a)
  count=0
  for i=1:size(a,1)
    count+=a[i]
   end
   return count
end

seq_red (generic function with 1 method)

In [53]:
sequential_sum=seq_red(h_a)

4993.198149163243

In [54]:
@test isapprox(sequential_sum,h_a1[size(h_a,1)])

Test Passed

In [55]:
@benchmark sequential_sum=seq_red(h_a)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     12.348 μs (0.00% GC)
  median time:      12.740 μs (0.00% GC)
  mean time:        13.222 μs (0.00% GC)
  maximum time:     33.333 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [56]:
@benchmark @cuda blocks=Int(ceil(n/1024)) threads=1024 parallel_red!(d_a)

BenchmarkTools.Trial: 
  memory estimate:  912 bytes
  allocs estimate:  34
  --------------
  minimum time:     8.989 μs (0.00% GC)
  median time:      10.655 μs (0.00% GC)
  mean time:        11.312 μs (0.00% GC)
  maximum time:     185.549 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1